# MxNet tutorial

In [6]:
from mxnet import nd
from mxnet.gluon import nn

### Create a neural network

In [7]:
net = nn.Sequential()

In [8]:
net.add(
    nn.Conv2D(channels=6, kernel_size=3, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120, activation="relu"),
    nn.Dense(84, activation="relu"),
    nn.Dense(10)
)
net

Sequential(
  (0): Conv2D(None -> 6, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (2): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (4): Dense(None -> 120, Activation(relu))
  (5): Dense(None -> 84, Activation(relu))
  (6): Dense(None -> 10, linear)
)

In [9]:
net.initialize()

x = nd.random.uniform(shape=(4, 1, 28, 28))
y = net(x)
y.shape

(4, 10)

#### 특정 Layer의 weight 및 bias 접근

In [10]:
# We can use [] to index a particular layer. For example, the following accesses the 1st layer’s weight and 6th layer’s bias.
(net[0].weight.data().shape, net[5].bias.data().shape)

((6, 1, 3, 3), (84,))

### Create a neural network flexibly
- `nn.Sequential`에서는 자동으로 forward path를 구성함.

In [11]:
class MixMLP(nn.Block):
    def __init__(self, **kwargs):
        # Run `nn.Block`'s init method
        super(MixMLP, self).__init__(**kwargs)
        self.blk = nn.Sequential()
        self.blk.add(nn.Dense(3, activation='relu'),
                     nn.Dense(4, activation='relu'))
        self.dense = nn.Dense(5)
    def forward(self, x):
        y = nd.relu(self.blk(x))
        print(y)
        return self.dense(y)

net = MixMLP()
net

MixMLP(
  (blk): Sequential(
    (0): Dense(None -> 3, Activation(relu))
    (1): Dense(None -> 4, Activation(relu))
  )
  (dense): Dense(None -> 5, linear)
)

In [12]:
net.initialize()
x = nd.random.uniform(shape=(2,2))
net(x)


[[0.00041689 0.00082987 0.00152172 0.        ]
 [0.00046854 0.00091932 0.00102066 0.        ]]
<NDArray 2x4 @cpu(0)>



[[ 1.3423392e-04  4.7004010e-05  1.1493427e-04  1.4829298e-04
  -3.9534425e-05]
 [ 1.1548035e-04  9.9556855e-06  9.7527067e-05  1.3528271e-04
  -5.6117766e-05]]
<NDArray 2x5 @cpu(0)>

In [13]:
net.blk[1].weight.data()


[[ 0.01904855  0.03239562 -0.03639716]
 [ 0.03737489 -0.04752456 -0.0686327 ]
 [ 0.04149481 -0.02683994  0.06428332]
 [-0.03739883 -0.00586057  0.00047984]]
<NDArray 4x3 @cpu(0)>